<a href="https://colab.research.google.com/github/sarahpursley/dsci551-assignment2/blob/main/q8_GROVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#generated emails 
!cd /content && rm -rf dsci551-assignment2 && git clone https://github.com/sarahpursley/dsci551-assignment2.git
import pandas as pd
import json 
import os
import sys

Cloning into 'dsci551-assignment2'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 322 (delta 19), reused 10 (delta 10), pack-reused 290
Receiving objects: 100% (322/322), 96.36 MiB | 22.10 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [ ]:
mal_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_mal.txt', mode = 'r').readlines()
phish_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_phish.txt', mode = 'r').readlines()
recon_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_recon.txt', mode = 'r').readlines()
se_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_se.txt', mode = 'r').readlines()


#using text generated from TTR algorithm for just the fraud message content
mal_fraud = open('dsci551-assignment2/datasets/generated_text/m_train.txt', mode = 'r').readlines()
se_fraud = open('dsci551-assignment2/datasets/generated_text/s_train.txt', mode = 'r').readlines()
recon_fraud = open('dsci551-assignment2/datasets/generated_text/r_train_.txt', mode = 'r').readlines()
phish_fraud = open('dsci551-assignment2/datasets/generated_text/p_train.txt', mode = 'r').readlines()

In [ ]:
# clean GPU text
generated_malware = [line for line in mal_gen if line.rstrip() != ''and line != '====================\n']
generated_phising = [line for line in phish_gen if line.rstrip() != ''and line != '====================\n']
generated_recon = [line for line in recon_gen if line.rstrip() != ''and line != '====================\n']
generated_se = [line for line in se_gen if line.rstrip() != ''and line != '====================\n']

In [ ]:
# create final jsonl file of messages to test
message_dict = {}
index = 0
for message in generated_malware:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'malware'}
  index+=1
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'malware'}
  index+=1

for message in generated_phising:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'phishing'}
  index+=1
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'phishing'}
  index+=1

for message in generated_recon:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'reconnaissance'}
  index+=1
  
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'reconnaissance'}
  index+=1

for message in generated_se:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'social_engineering'}
  index+=1
  
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'social_engineering'}
  index+=1

#real human emails
for message in se_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'social_engineering'}
  index+=1

for message in mal_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'malware'}
  index+=1

for message in recon_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'reconnaissance'}
  index+=1

for message in phish_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'phishing'}
  index+=1

#example output
message_dict['train_emails_0']


{'email': "AIM OF CONFIDENTIALITY;IF YOU ARE INTERESTED IN THIS TRANSACTION PLEASE CONTACT ME IMMEDIATELY;I am contacting you so that you can  learn more about this transaction  and act as a guardian.  I am an accountant and the head of an  accountant's office in the accounts committee in  the senate of the senate of the Federal Republic Of Nigeria.  Prior to this present government, my husband had his personal attorney  who was specially appointed to represent him in this transaction, however, after due consultation  with key members of the committee, the client has now decided to take  his own life.  I am contacting you to assist in repatriating the money and property  left behind by my husband before it is confiscated or declared  unserviceable by the bank where these huge deposits were lodged.  Particularly, the bank where these deposits were lodged has issued me a notice to provide the  company with more documents and the release codes of\n",
 'email_type': 'malware',
 'label': 'm

In [ ]:
print("There are this many 'lines': " + str(index))

There are this many 'lines': 5290


In [ ]:
# read in github for grover and install requirements
!cd /content && rm -rf grover && git clone https://github.com/rowanz/grover.git
%cd /content/grover

!pip install cuda
!pip install keras
!pip install -r requirements-gpu.txt
!python download_model.py base

Cloning into 'grover'...
remote: Enumerating objects: 104, done.
remote: Total 104 (delta 0), reused 0 (delta 0), pack-reused 104
Receiving objects: 100% (104/104), 674.41 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/grover
  Using cached https://files.pythonhosted.org/packages/0f/39/bdd75b08a6fba41f098b6cb091b9e8c7a80e1b4d679a581a0ccd17b10373/tensorboard-1.13.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2c/65/8dc8fc4a263a24f7ad935b72ad35e72ba381cb9e175b6a5fe086c85f17a7/tensorflow_gpu-1.13.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bb/48/13f49fc3fa0fdf916aa1419013bb8f2ad09674c275b4046d5ee669a46873/tensorflow_estimator-1.13.0-py2.py3-none-any.whl
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
  

In [ ]:
# FOR THE FIRST TIME YOU MUST UNINSTALL AND REINSTALL TENSORFLOW/TENSORFLOW GPU
# only do this for the first runtime
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 377.1MB 44kB/s 
     |████████████████████████████████| 491kB 9.3MB/s 
     |████████████████████████████████| 3.2MB 37.9MB/s 
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-gpu 1.13.1
    Uninstalling tensorflow-gpu-1.13.1:
      Successfully uninstalled tensorflow-gpu-1.13.1


In [ ]:
import json
!rm -rf input_data && mkdir input_data

all_emails = message_dict

with open("input_data/train_data.jsonl", 'w') as f:
    for item in all_emails.values():
        f.write(json.dumps(item) + "\n")
    print('Training data exported to input_data as train_data.jsonl')

Training data exported to input_data as train_data.jsonl


In [ ]:
!export PYTHONPATH=$(pwd)
! PYTHONPATH=$(pwd) python lm/train.py --config_file lm/configs/base.json --input_file input_data/train_data.jsonl --num_train_steps 10000 --output_dir /content/grover

INFO:tensorflow:*** Input Files ***
INFO:tensorflow:  input_data/train_data.jsonl

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/content/grover', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5c2a8235d0>, '_task_type': 'worker', '_task_id'

In [ ]:
!zip -r /content/grover_train_output.zip /content/grover/checkpoint
!zip -r /content/grover_train_output.zip /content/grover/model.ckpt-0.data-00000-of-00001
!zip -r /content/grover_train_output.zip /content/grover/model.ckpt-0.index
!zip -r /content/grover_train_output.zip /content/grover/model.ckpt-0.meta

  adding: content/grover/checkpoint (deflated 42%)
  adding: content/grover/model.ckpt-0.data-00000-of-00001 (deflated 8%)
  adding: content/grover/model.ckpt-0.index (deflated 80%)
  adding: content/grover/model.ckpt-0.meta (deflated 91%)


In [38]:
from google.colab import files
files.download('/content/grover_train_output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Discrimination Code...will not run after testing various parameters, see example output of what we are getting as results**
-  This could be due to either the model checkpoints or training data format
- Kept getting either 'EXITING BECAUSE DO TRAIN is true' or 'EXITING BECAUSE NO CHECKPOINT.;


In [ ]:
%cd /content/grover
!export PYTHONPATH=$(pwd)
! PYTHONPATH=$(pwd) python discrimination/run_discrimination.py --additional_data True --input_data input_data/train_data.jsonl --init_checkpoint grover/model.ckpt-0 --config_file lm/configs/base.json --output_dir content/grover --num_train_epochs 10.0 

/content/grover
The output directory content/grover exists!
EXITING BECAUSE NO CHECKPOINT.
